In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
import numpy as np
from sympy import *

from sympy.interactive import printing
printing.init_printing(use_latex='mathjax')

## volcanic eruptions and climate change

In [4]:
# stefan-boltzmann constant
sigma_b = 5.6696e-8 # Wm^(-2)K^(-4)

In [5]:
# todo: put regional parameters in config files

# regional parameters
# tau = 1 # atmospheric transmissivity
# alpha_sky = np.ndarray()  # top of atmosphere albedos
# alpha_surface = np.ndarray()  # surface albedos
# gamma = np.ndarray()  # spatial radiation factors
# rho = np.ndarray()  # densities
# epsilon = np.ndarray()  # emissivities
# specific_heats = np.ndarray()  # specific heat capacities (c_k)


#### regional temperature variation

In [8]:
t, phi = symbols('t varphi')
T = Function("T_k")(t, phi)
S = Function("S")(t)
d_temp = T.diff(t)
c, A, Z = symbols("c_k A_k Z_k")
alpha, gamma, rho, tau, sigma, epsilon = symbols("alpha_k gamma rho_k tau sigma_B varepsilon")


In [9]:
diff_eq = Eq(c*rho*Z*d_temp, gamma*(1 - alpha)*S - epsilon*tau*sigma*T**4)
diff_eq

         ∂                                                        4           
Zₖ⋅cₖ⋅ρₖ⋅──(Tₖ(t, varphi)) = γ⋅(1 - αₖ)⋅S(t) - σ_B⋅τ⋅varepsilon⋅Tₖ (t, varphi)
         ∂t                                                                   

In [33]:
# pseudocode
def compute_couplings(temperatures, transfer_coefficients, zonal_lengths):
    # todo: figure out how to do this as matrix multiplication or dot product
    return transfer_coefficients*zonal_lengths*(temperatures - temperatures)

def solar_radiative_flux(time):
    radiative_flux = None  # what form? linear or exponential
    return radiative_flux

In [40]:
def dtemperature_dt(time: float, temperatures: np.ndarray, zonal_lengths: np.ndarray,
                     transfer_coefficients: np.ndarray, albedo_sky: np.ndarray, albedo_surface: np.ndarray,
                     solar=None, volcano_model=None, kwargs=None):
    """
    :param time: scalar
    :param temperatures: temperature field
    :param transfer_coefficients: pre-computed heat transfer coefficients
    :param albedo_surface:
    :param albedo_sky:
    :param solar: solar flux model (maybe constant modulated by incident angle?)
    :param volcano_model: TBD
    :param kwargs: for scipy.integrators.solve_ivp
    :param zonal_lengths:
    :return: temperature changes for each zone
    """
    # numerical solve equation 8
    coupling_terms = compute_couplings(temperatures, transfer_coefficients, zonal_lengths)

    # precompute coupled terms using itertools or product
    # precompute each term (they're all six-component vectors)
    flux_in = gamma*(1-albedo_sky)*(1-albedo_surface)*solar(time) + coupling_terms
    flux_out = epsilon*tau*sigma_b*T**4

    dtemperature = flux_in - flux_out  # solve regions together

    return dtemperature

In [ ]:
# todo: compute coupling terms L_ij*k_ij*(T_j - T_i) [how to vectorize: use itertools, handling top and bottom layers manually]
# todo: make function solar for solar radiative flux

In [ ]:
# todo: compute steady state, no coupling
# todo: compute steady state with coupling